In [1]:
from datetime import datetime, timedelta
from selenium import webdriver
import time
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [2]:
first_part_url = "https://www.momondo.com/flight-search/"
last_part_url = "/"

In [3]:
TTT = [(datetime.today() + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(0, 30)]

In [4]:
LOS = [(datetime.today() + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(1, 6)]

In [5]:
snapshot_date = datetime.today().strftime('%Y-%m-%d')
snapshot_date

'2025-03-02'

In [6]:
dep = ['LON' , 'PAR' , 'ROM' ]
back = ['LON' , 'PAR' , 'ROM' ]

In [7]:
all_urls = []

# Outer loops over the city combinations
for origin in dep:
    for destination in back:
        if origin != destination:
            # For a given city pair, iterate over all departure dates
            for t in TTT:
                # For each departure date, create return dates offset from +1 to +5 days
                for delta in range(1, 6):
                    return_date = (datetime.strptime(t, "%Y-%m-%d") + timedelta(days=delta)).strftime("%Y-%m-%d")
                    # Construct the URL including departure and return dates
                    url = f"{first_part_url}{origin}-{destination}/{t}/{return_date}{last_part_url}"
                    all_urls.append((url, t, return_date))

# Display the total count and a few sample URLs
print(f"Total URLs generated: {len(all_urls)}")
print("Sample URLs:")
for u in all_urls[:]:
    print(u)

Total URLs generated: 900
Sample URLs:
('https://www.momondo.com/flight-search/LON-PAR/2025-03-02/2025-03-03/', '2025-03-02', '2025-03-03')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-02/2025-03-04/', '2025-03-02', '2025-03-04')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-02/2025-03-05/', '2025-03-02', '2025-03-05')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-02/2025-03-06/', '2025-03-02', '2025-03-06')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-02/2025-03-07/', '2025-03-02', '2025-03-07')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-03/2025-03-04/', '2025-03-03', '2025-03-04')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-03/2025-03-05/', '2025-03-03', '2025-03-05')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-03/2025-03-06/', '2025-03-03', '2025-03-06')
('https://www.momondo.com/flight-search/LON-PAR/2025-03-03/2025-03-07/', '2025-03-03', '2025-03-07')
('https://www.momondo.com/flight-search/LON-PAR/2025

In [8]:
driver = webdriver.Chrome()
driver.get(all_urls[0][0])

In [9]:
while True:
    elements = driver.find_elements(By.XPATH, './/*[@class="Fxw9-result-item-container"]')
    if len(elements) >= 100:
        break
    try: #Finding "Show more results" button.
                load_more_button = WebDriverWait(driver,45).until(
                    EC.element_to_be_clickable((By.XPATH,"//div[div[contains(text(),'Show more results')]]"))
                )
                load_more_button.click()
                time.sleep(1) # small delay to allow results to load
    except:
        break
len(elements)

101

In [10]:
for item in driver.find_elements(By.XPATH, './/*[@class=\"nrc6-wrapper"]'):
        duration = item.find_element(By.XPATH, '//*[@class=\"nrc6-content-section"]').text
        price = item.find_element(By.XPATH, './/*[@class=\"f8F1-price-text"]').text
        print (duration, price)

Save
Share
Best
6:30 pm – 8:50 pm
easyJet
nonstop
1h 20m
LGW
-
CDG
7:20 am – 7:30 am
easyJet
nonstop
1h 10m
CDG
-
LGW $217
Save
Share
Best
6:30 pm – 8:50 pm
easyJet
nonstop
1h 20m
LGW
-
CDG
7:20 am – 7:30 am
easyJet
nonstop
1h 10m
CDG
-
LGW $209
Save
Share
Best
6:30 pm – 8:50 pm
easyJet
nonstop
1h 20m
LGW
-
CDG
7:20 am – 7:30 am
easyJet
nonstop
1h 10m
CDG
-
LGW $217
Save
Share
Best
6:30 pm – 8:50 pm
easyJet
nonstop
1h 20m
LGW
-
CDG
7:20 am – 7:30 am
easyJet
nonstop
1h 10m
CDG
-
LGW $222
Save
Share
Best
6:30 pm – 8:50 pm
easyJet
nonstop
1h 20m
LGW
-
CDG
7:20 am – 7:30 am
easyJet
nonstop
1h 10m
CDG
-
LGW $222
Save
Share
Best
6:30 pm – 8:50 pm
easyJet
nonstop
1h 20m
LGW
-
CDG
7:20 am – 7:30 am
easyJet
nonstop
1h 10m
CDG
-
LGW $214
Save
Share
Best
6:30 pm – 8:50 pm
easyJet
nonstop
1h 20m
LGW
-
CDG
7:20 am – 7:30 am
easyJet
nonstop
1h 10m
CDG
-
LGW $228
Save
Share
Best
6:30 pm – 8:50 pm
easyJet
nonstop
1h 20m
LGW
-
CDG
7:20 am – 7:30 am
easyJet
nonstop
1h 10m
CDG
-
LGW $229
Save
Share
Best


In [ ]:
with open('flight_data_momondo.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Snap Date","Dep Date","Return Date","Dep time","Dep Time","Dep Duration","From","Back time","Back Comp","Stops","Back Time","Back Duration","Back Arrival","From","Stops","Landing","Price"])

    for url, dep_date, return_date in all_urls:
        driver.get(url)
        time.sleep(4)
        
        # Click 'Show More' button until 100 results are present
        while True:
            elements = driver.find_elements(By.XPATH, './/*[@class="Fxw9-result-item-container"]')
            if len(elements) >= 100:
                break
            try: #Finding "Show more results" button.
                        load_more_button = WebDriverWait(driver,45).until(
                            EC.element_to_be_clickable((By.XPATH,"//div[div[contains(text(),'Show more results')]]"))
                        )
                        load_more_button.click()
                        time.sleep(1) # small delay to allow results to load
            except:
                break #exit loop if the button is missing or unclickable.

        
        # Extracting flight details
        for item in driver.find_elements(By.XPATH,  './/*[@class="Fxw9-result-item-container"]'):
            duration_dep = WebDriverWait(driver,4).until(EC.visibility_of_element_located((By.XPATH,"//div[contains(@class,'nrc6-inner')]")))
            duration_parts = duration_dep.text.split("\n")  # Splitting by newline to commas.
            #Removing "Throwaway ticketing hack" | bags | cabins if it exists. הייתה פה חשיבה מחוץ לקופסא
            duration_parts = [part for part in duration_parts if part.strip() not in ("Save",0,"0",1,"1","-","Share","Ad","Best","Cheapest","View Deal","Mix & Match")]

            #image_element = driver.find_element(By.XPATH, './/*[@class="max-w-none bg-transparent rounded-100 last:self-end h-icon-large w-icon-large"]')
            #dep_back_companies = image_element.get_attribute("title")
            #companies_parts = dep_back_companies.split("\n")
            #price = item.find_element(By.XPATH, './/*[@class="Fxw9-result-item-container"]')
            #tag = item.find_element(By.XPATH,'.//*[@class=\"f8F1-price-text"]').text
            writer.writerow([snapshot_date,dep_date, return_date] + duration_parts)

# Close WebDriver
driver.quit()